In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
from ipywidgets import widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import warnings
warnings.filterwarnings("ignore")

In [8]:
class eclairys:
    def __init__(self, stock_ticker, sector_ticker, market_ticker, short_period=[5, 10, 20], long_period=[50, 100, 200]):
        self.stock_ticker = stock_ticker
        self.sector_ticker = sector_ticker
        self.market_ticker = market_ticker
        self.short_period = short_period
        self.long_period = long_period
        self.period_list = short_period + long_period
        self.stock_history = None
        self.market_history = None
        self.sector_history = None
        self.market_relative_history = None
        self.sector_relative_history = None

    def fetch_data(self):
        """Fetch historical data for the stock and market."""
        self.stock_history = yf.Ticker(self.stock_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.stock_history.name = self.stock_ticker
        self.market_history = yf.Ticker(self.market_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.market_relative_history = self.stock_history / self.market_history
        self.sector_history = yf.Ticker(self.sector_ticker).history('max')['Close'].tz_localize(None).dropna()
        self.sector_relative_history = self.stock_history / self.sector_history

    def calculate_moving_averages(self, data):
        """Calculate moving averages and moving average differentials."""
        MA_df, MAD_df = pd.DataFrame(), pd.DataFrame()
        for period in self.period_list:
            if len(data) < period:
                MA_df[f'MA {period}'] = np.nan
                MAD_df[f'MAD {period}'] = np.nan
            else:
                MA = data.rolling(period).mean()
                MA_df[f'MA {period}'] = MA
                MAD = MA.diff(5)
                MAD_df[f'MAD {period}'] = MAD
        return MA_df, MAD_df

    def normalize_mad(self, MAD_df):
        """Normalize MAD using a rolling window."""
        MADN_df = pd.DataFrame(index=MAD_df.index)
        rolling_max = MAD_df.rolling(400, min_periods=1).max()
        rolling_min = MAD_df.rolling(400, min_periods=1).min()
        MADN_df = 100 * (MAD_df - rolling_min) / (rolling_max - rolling_min)
        MADN_df = MADN_df.replace([np.inf, -np.inf], 0).fillna(0)
        return MADN_df

    def calculate_grade(self, MADN_df):
        """Calculate the GRADE score as the mean of normalized MAD values."""
        GRADE = MADN_df.mean(axis=1)
        return GRADE

    def generate_gps(self, MAD_df, short_period, long_period):
        """Generate GPS indicators based on booleanized MAD values."""
        BMAD = (MAD_df >= 0).astype(int)
        
        # Créer les noms de colonnes pour les périodes courtes et longues
        short_columns = [f'MAD {p}' for p in short_period]
        long_columns = [f'MAD {p}' for p in long_period]
        
        # Calculer la somme des périodes courtes et longues
        SBMAD = BMAD[short_columns].sum(axis=1)
        LBMAD = BMAD[long_columns].sum(axis=1)

        # Générer les indicateurs GPS en fonction des conditions
        conditions = [
            (SBMAD >= 2) & (LBMAD >= 2),
            (SBMAD < 2) & (LBMAD < 2),
            (SBMAD >= 2) & (LBMAD < 2),
            (SBMAD < 2) & (LBMAD >= 2)
        ]
        choices = ['A', 'B', 'C', 'P']
        GPS = pd.Series(np.select(conditions, choices, default=None), index=MAD_df.index)
        return GPS

    def run_analysis(self):
        """Run the entire analysis and return the output DataFrame."""
        self.fetch_data()
        
        # Calculate indicators for the stock
        stock_MA_df, stock_MAD_df = self.calculate_moving_averages(self.stock_history)
        stock_MADN_df = self.normalize_mad(stock_MAD_df)
        stock_GRADE = self.calculate_grade(stock_MADN_df)
        stock_GPS = self.generate_gps(stock_MAD_df, self.short_period, self.long_period)

        # Combine stock indicators into a DataFrame
        stock_output = pd.concat([
            pd.DataFrame(self.stock_history),
            stock_MA_df,
            pd.DataFrame(stock_GRADE, columns=['GRADE']),
            pd.DataFrame(stock_GPS, columns=['Absolute GPS'])
        ], axis=1)

        # Calculate indicators for the relative trend on sector
        sector_MA_df, sector_MAD_df = self.calculate_moving_averages(self.sector_relative_history)
        sector_GPS = self.generate_gps(sector_MAD_df, self.short_period, self.long_period)
        stock_output['Sector GPS'] = sector_GPS

        # Calculate indicators for the relative trend on market
        market_MA_df, market_MAD_df = self.calculate_moving_averages(self.market_relative_history)
        market_GPS = self.generate_gps(market_MAD_df, self.short_period, self.long_period)
        stock_output['Market GPS'] = market_GPS

        return stock_output

In [10]:
# Exemple d'utilisation de eclairys v4
eclairys('BA', '^SP500-20', '^GSPC').run_analysis()

,BA,MA 5,MA 10,MA 20,MA 50,MA 100,MA 200,GRADE,Absolute GPS,Sector GPS,Market GPS
Date,,,,,,,,,,,
1962-01-02,0.190931,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1962-01-03,0.194750,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1962-01-04,0.192840,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1962-01-05,0.189022,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
1962-01-08,0.189499,0.191408,NaN,NaN,NaN,NaN,NaN,0.000000,B,B,B
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06,147.160004,151.426001,152.529999,153.312500,156.1116,167.1637,177.53805,41.794527,B,B,B
2024-11-07,150.979996,151.760001,152.107999,153.530500,155.7170,166.8896,177.22230,43.091212,B,B,B
2024-11-08,151.679993,151.178000,151.774998,153.563499,155.3054,166.6565,176.97130,43.221336,B,B,B


In [22]:
# Function to fetch and process data
def fetch_data(stock_ticker, sector_ticker, market_ticker):
    df = eclairys(stock_ticker, sector_ticker, market_ticker).run_analysis()
    df = df.iloc[-200:]
    df = df.resample('D').ffill()
    return df

# Function to plot the data
def plot_data(stock_ticker, sector_ticker, market_ticker, df):
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(14, 12), sharex=True, gridspec_kw={'height_ratios': [3, 1, 0.25, 0.25, 0.25], 'hspace': 0.2})

    # 1st plot: Stock Price and Moving Averages
    ax1.plot(df.index, df[stock_ticker], label=stock_ticker, color='black', linewidth=1)
    ax1.plot(df.index, df['MA 5'], label='MA 5', linestyle='--', color='blue', linewidth=0.85)
    ax1.plot(df.index, df['MA 10'], label='MA 10', linestyle='--', color='orange', linewidth=0.85)
    ax1.plot(df.index, df['MA 20'], label='MA 20', linestyle='--', color='green', linewidth=0.85)
    ax1.plot(df.index, df['MA 50'], label='MA 50', linestyle='--', color='red', linewidth=0.85)
    ax1.plot(df.index, df['MA 100'], label='MA 100', linestyle='--', color='purple', linewidth=0.85)
    ax1.plot(df.index, df['MA 200'], label='MA 200', linestyle='--', color='brown', linewidth=0.85)
    ax1.legend(loc='upper left')
    ax1.set_title(f'{stock_ticker} Price and Moving Averages')
    ax1.grid(True)

    # 2nd plot: GRADE
    ax2.plot(df.index, df['GRADE'], color='black')
    ax2.axhline(y=75, color='green', linewidth=0.75)
    ax2.axhline(y=25, color='red', linewidth=0.75)
    ax2.set_title('GRADE')
    ax2.grid(True)

    # 3rd plot: Absolute GPS with color bands
    gps_colors = {'A': 'green', 'B': 'red', 'P': 'blue', 'C': 'orange'}
    for grade, color in gps_colors.items():
        mask = df['Absolute GPS'] == grade
        ax3.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax3.get_xaxis_transform())
    ax3.set_title('Absolute GPS')
    ax3.set_yticks([])

    # 4th plot: Market GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['Sector GPS'] == grade
        ax4.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax4.get_xaxis_transform())
    ax4.set_title('Sector GPS')
    ax4.set_yticks([])

    # 5th plot: Market GPS with color bands
    for grade, color in gps_colors.items():
        mask = df['Market GPS'] == grade
        ax5.fill_between(df.index, 0, 1, where=mask, color=color, alpha=0.5, transform=ax5.get_xaxis_transform())
    ax5.set_title('Market GPS')
    ax5.set_yticks([])

    # Add GPS legend
    legend_elements = [Patch(facecolor='green', label='A', alpha=0.5),
                       Patch(facecolor='red', label='B', alpha=0.5),
                       Patch(facecolor='blue', label='P', alpha=0.5),
                       Patch(facecolor='orange', label='C', alpha=0.5)]
    ax3.legend(handles=legend_elements, loc='upper left', title='GPS Categories')

    plt.tight_layout()
    plt.show()

# Widgets for interaction
stock_ticker_widget = widgets.Text(value='BA', description='Stock:')
sector_ticker_widget = widgets.Text(value='^SP500-20', description='Sector:')
market_ticker_widget = widgets.Text(value='^GSPC', description='Market:')
run_button = widgets.Button(description='Run Analysis', button_style='success')

# Output widget to display the plot
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        stock_ticker = stock_ticker_widget.value
        sector_ticker = sector_ticker_widget.value
        market_ticker = market_ticker_widget.value
        df = fetch_data(stock_ticker, sector_ticker, market_ticker)
        plot_data(stock_ticker, sector_ticker, market_ticker, df)

# Bind the button click event
run_button.on_click(on_button_click)

# Display the widgets
display(stock_ticker_widget, sector_ticker_widget, market_ticker_widget, run_button, output)


Text(value='BA', description='Stock:')

Text(value='^SP500-20', description='Sector:')

Text(value='^GSPC', description='Market:')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Output()